In [ ]:
import os
import numpy as np
import warnings
from astropy.io import fits
from astropy.modeling import models, fitting
import matplotlib.pyplot as plt
import glob
import math
%matplotlib inline

In [ ]:
fit_g = fitting.LevMarLSQFitter()
TEST_DIR = '../test_data/'

In [ ]:
def plot_std_on_curve(total_data, data_set, time_list, label_list):
    display_color = ['y', 'r', 'k', 'g', 'y', 'm', 'c'] # set the color to'w' to hide the display of any data set
    
    plt.figure(figsize=(8,8))
    
    for n in range(total_data):
        label_curve = label_list[n]
        plt.plot(time_list, data_set[n, :], display_color[n]+'o', label=label_list[n])
    
    plt.legend(loc="upper left", prop={'size': 12})
    plt.xlabel('Times [Days]')
    plt.ylabel('RV [m/s]')
    ymax = math.ceil(np.amax(data_set))
    ymin = math.floor(np.amin(data_set))
    plt.ylim((ymin, ymax))

In [ ]:
def fit_ccf(velocities, ccf, rv_guess, velocity_cut=100.0):
    g_init = models.Gaussian1D(amplitude=-1e7, mean=rv_guess, stddev=5.0)
    i_cut = (velocities >= rv_guess - velocity_cut) & (velocities <= rv_guess + velocity_cut)
    g_x = velocities[i_cut]
    g_y = ccf[i_cut]-np.nanmedian(ccf)
    gaussian_fit = fit_g(g_init, g_x, g_y)
    rv_result = gaussian_fit.mean.value
    return gaussian_fit, rv_result, g_x, g_y


In [ ]:
def get_harps_ccf(keystr):
    #import pdb; pdb.set_trace()
    harps_file =  TEST_DIR+'rv_test/HARPStauceti_baseline/ccf/HARPS'+keystr+'_ccf_G2_A.fits'
    harps_ccf, harps_ccf_head = fits.getdata(harps_file, header=True)
    return harps_ccf, harps_ccf_head

In [ ]:
def get_other_ccf(keystr):
    #other_file = TEST_DIR+'rv_test/output_cindy_fix/HARPStaucetiCindy' + keystr + '_norm_ccf_G2_A.fits'
    other_file = TEST_DIR + 'rv_test/output_cindy_03232020/HARPStaucetiCindy' + keystr + '_norm_ccf_G2_A.fits'
    other_ccf, other_ccf_head = fits.getdata(other_file, header=True)
    return other_ccf, other_ccf_head

## Compare Arpita output with reference output

In [ ]:
rv_output_arpita_fits = glob.glob(TEST_DIR+'rv_test/output_arpita/*.fits')
rv_output_arpita_fits.sort()
total_fits = len(rv_output_arpita_fits)
print(total_fits)
times_array = np.zeros(total_fits, dtype=float)
means = np.zeros((3, total_fits), dtype=float)

for n in range(total_fits):
    fits_name = rv_output_arpita_fits[n]   
    i1 = fits_name.index('ARPITA.20')+6

    harps_ccf, harps_ccf_head = get_harps_ccf(fits_name[i1:i1+24])
    times_array[n] = harps_ccf_head['MJD-OBS']
    
    rv_data, rv_head = fits.getdata(fits_name, header=True)
    other_ccf, other_ccf_head = get_other_ccf(fits_name[i1:i1+24])

    s = np.shape(rv_data)
    v_size = s[0]
    vel_sum = rv_data[v_size-1, :]
    vel_loop = rv_data[v_size-2, :]
    rv_fit, rv_result, g_x, g_y = fit_ccf(vel_loop, vel_sum, vel_loop[int(s[1]/2)])
 
    harps_mean = harps_ccf_head['HIERARCH ESO DRS CCF RVC']
    other_mean = other_ccf_head['CCF-RVC']
    
    means[0][n] = harps_mean
    means[1][n] = rv_result
    means[2][n] = other_mean


min_time = np.amin(times_array)
times_0 = times_array - min_time

mean_harps = np.mean(means[0, :])
mean_rv = np.mean(means[1, :])
mean_other = np.mean(means[2, :])

#print('mean: ', mean_harps, mean_rv, mean_other)

offset_rv = np.zeros((3, times_array.size), dtype=float)
offset_rv[0, :] = (means[0, :] - mean_harps) * 1000.0
offset_rv[1, :] = (means[1, :] - mean_rv) * 1000.0
offset_rv[2, :] = (means[2, :] - mean_other) * 1000.0

std_label = list()
std_label.append('HARPS Sigma =' + str(np.std(offset_rv[0, :])) + ' m/s')
std_label.append('Arpita Sigma =' + str(np.std(offset_rv[1, :])) + ' m/s')
std_label.append('Cindy Sigma =' + str(np.std(offset_rv[2, :])) + ' m/s')

#import pdb; pdb.set_trace()
plot_std_on_curve(3, offset_rv, times_0, std_label)